A web crawler is set up to pull sale data off of p24. 

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from tqdm import tqdm 

# Define the number of iterations
num_iterations = 1

pricelist = []
locationlist = []

# Create a tqdm progress bar
for x in tqdm(range(num_iterations), desc="Scraping Progress"):
    p24 = f"https://www.property24.com/houses-for-sale/gauteng/1/p{x}"
    req = requests.get(p24)
    soup = BeautifulSoup(req.text, "html.parser")

    for location in soup.find_all("span", attrs=["p24_location"]):
        locationlist.append(location.text)

    for price in soup.find_all("span", attrs=["p24_price"]):
        pricelist.append(price.text)
    
    for location in soup.find_all("div", class_="p24_location"):
        locationlist.append(location.text)

    for location in soup.find_all("div", class_="p24_price"):
        locationlist.append(location.text)
        

pricelist = [price.strip().replace("\n", "").replace("\r", "").replace(" ", "").replace("R", "").replace("POA", "0") for price in pricelist]    

csvdata = [[addr, price] for addr, price in set(zip(pricelist, locationlist))]

print(f"CSVDATA: {len(csvdata), csvdata}")

Using the propattr class the median and sd are calculated for the data. 

The data should be sorted by area name to avoid over use of the api. 

The data is sorted by name and pulled into its own list to find the co-ordinates using Bings Map Api.

In [ ]:
import geocoder
from tqdm import tqdm

commonlocations = []


for data in csvdata:
    if data[1] not in commonlocations:
        commonlocations.append(data[1])

print(f"SIZE: {len(commonlocations)} {commonlocations}")

common_coords = []

failed = []

for addr in tqdm(commonlocations, desc="Finding Coordinates"):
    try:
        g = geocoder.bing(f'{addr + " " + "South Africa"}', key="Au_zOIgH37lhBFI2w8tNQxjZRxFwuD7892Hx7mBZV6ifLba99oxRQFh6uryjZ5cD")
        
        # Check if the geocoding request was successful
        if g.ok:
            results = g.json
            common_coords.append([results["lat"], results["lng"], addr])
        else:
            # Handle the case where the geocoding request failed
            print(f"Geocoding request failed for address: {addr}")
            failed.append(addr)
    except Exception as e:
        # Handle any exceptions that may occur during the geocoding request
        print(f"An error occurred while geocoding address: {addr}")
        print(f"Error details: {str(e)}")

Enter data into csv for locations, price and coords.

In [ ]:
with open(f"prop_gauteng_locations_price.csv",  "w") as r:
    fieldnames = ["Location", "Price"]
    writer = csv.writer(r)
    for row in csvdata:
        writer.writerow(row)

with open("gauteng_area_coords.csv", "w") as w:
    writer = csv.writer(w)
    for row in common_coords:
        writer.writerow(row)